In [1]:
from pathlib import Path
from common.data.amigos.utils import load_participant_data
from common.data.amigos.config import AmigosConfig
import mne.io

from common.data.eeg.saliency_extractor import EEGFeatureExtractor

info = mne.create_info(
    ch_names=AmigosConfig.CH_NAMES,
    ch_types=AmigosConfig.CH_TYPES,
    sfreq=AmigosConfig.original_eeg_fs
)
participant_data = load_participant_data(Path("../../../resources/AMIGOS/pre_processed_py/"))
eeg_data = participant_data["P01"]["joined_data"][0]
raw = mne.io.RawArray(eeg_data.T, info=info, verbose=False)

In [ ]:
extractor = EEGFeatureExtractor(raw)
starts = extractor.pick_segments(
    4, 0.125, bands=((4, 8), (8, 13), (13, 30)), band_weights=(0.4, 0.5, 0.4)
)

In [2]:
from common.data.sampler import FeatureAndRandomLogUniformIntervalsSegmenter

feat_seg = FeatureAndRandomLogUniformIntervalsSegmenter(
    min_length=1,
    max_length=30,
    num_segments=20,
    extraction_jitter=0,
    anchor_identification_hop=0.125
)

In [3]:
from common.data.eeg import EEG
import numpy as np

r = feat_seg.compute_segments(EEG(data=raw, eid="1", fs=128))
a = np.array(r)

np.diff(a)

Finding segments below or above PTP threshold.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 57 samples (0.445 s)

Setting up low-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 213 sam

array([[ 4.       ],
       [ 4.       ],
       [ 1.828125 ],
       [ 4.       ],
       [ 2.6015625],
       [ 4.       ],
       [ 4.       ],
       [ 3.4375   ],
       [ 5.078125 ],
       [ 6.34375  ],
       [ 7.2578125],
       [ 7.296875 ],
       [ 7.6328125],
       [ 9.078125 ],
       [10.2890625],
       [14.1640625],
       [18.6171875],
       [21.078125 ],
       [21.3828125]])

In [6]:
a

array([[56.625    , 60.625    ],
       [86.75     , 90.75     ],
       [39.3203125, 41.1484375],
       [31.125    , 35.125    ],
       [61.5234375, 64.125    ],
       [26.5      , 30.5      ],
       [65.375    , 69.375    ],
       [44.03125  , 47.46875  ],
       [15.7578125, 20.8359375],
       [68.0078125, 74.3515625],
       [35.40625  , 42.6640625],
       [50.609375 , 57.90625  ],
       [ 7.4453125, 15.078125 ],
       [17.34375  , 26.421875 ],
       [39.8359375, 50.125    ],
       [21.1796875, 35.34375  ],
       [63.7890625, 82.40625  ],
       [35.4921875, 56.5703125],
       [14.6015625, 35.984375 ]])

In [4]:
# Total coverage:
unique_coverage = 0
duration = raw.duration

taken_quarter_seconds = np.array([0 for i in range(int(duration * 4))])
for start, stop in a:
    start = int(start * 4)
    stop = int(stop * 4)
    taken_quarter_seconds[start:stop] += 1

unique_coverage = len(taken_quarter_seconds[taken_quarter_seconds > 0]) / (raw.duration * 4)
print(unique_coverage)  # We use 95% of the original signal

0.8166871165644172


In [5]:
# Coverage of SHORT segments:
unique_coverage = 0
duration = raw.duration

taken_quarter_seconds = np.array([0 for i in range(int(duration * 4))])
for start, stop in a[a[:, 1] - a[:, 0] < 4.1]:
    start = int(start * 4)
    stop = int(stop * 4)
    taken_quarter_seconds[start:stop] += 1

unique_coverage = len(taken_quarter_seconds[taken_quarter_seconds > 0]) / (raw.duration * 4)
print(unique_coverage)  # We use 95% of the original signal

0.2879345603271984
